## Tutorial on enzyme design with ProteinMPNN and data analysis of generated sequences

### 1. Problem statement: 

- We are given an enzyme with a bound cofactor in the active site. We want to redesign (and diversify) its sequence while retaining active site residues from native. How can we achieve it? The most straightforward option would be **ProteinMPNN** (doi: 10.1126/science.add21) - a message passing graph neural network that has been shown remarkable inverse folding modelling accuracy. In this tutorial we will try to understand how to use ProteinMPNN and analyze generated data relying on several scripts.

### 2. Requirements for the tutorial: 

- The following tutorial requires basic knowledge of python, understanding of ProteinMPNN and ESMfold (or AlphaFold2). The tutorial doesn't serve a purpose to explain how all the mentioned software works. 

- In order to run ProteinMPNN, you should have installed and configured a conda environment (see https://github.com/dauparas/ProteinMPNN/tree/main). To predict protein structures of designed sequences, ESMfold should be installed (https://github.com/facebookresearch/esm). 

- If installation is not possible, use the webservers of ProteinMPNN (https://huggingface.co/spaces/simonduerr/ProteinMPNN) and ESMfold (https://esmatlas.com/resources?action=fold). ESMfold has a size limitation of 400 aa residues! 

### 3. Protein of interest for input
- Input to ProteinMPNN is a structural file (.pdb) of a protein of interest. ProteinMPNN processes pdbs and ignores water or ions, so no pre-processing has to be carried out. Though I would recommend deleting all unneccessary HETATMs. 

- In the tutorial we will work with HotAldO (https://www.uniprot.org/uniprotkb/A0LST6) - an FAD-dependent monooxygenase catalyzing oxidation of 1,2-diols. It is a monomeric enzyme of 46 kDa molecular weight; there is no crystal/cryoEM structure available for HotAldO. However, a crystal structure of a close homologue AldO (https://www.uniprot.org/uniprotkb/Q9ZBU1) is available with bound FAD and a substrate xylose (https://www.rcsb.org/structure/2VFS). This important information will help us in a bit.

- Let's visualize AldO! We will visualize it in py3dmol in this notebook. If you don't have py3dmol installed, run the following cell in your terminal: 

In [1]:
pip install py3Dmol

  Obtaining dependency information for py3Dmol from https://files.pythonhosted.org/packages/d0/06/8b41d1dd0ac73d653c76f68563a8a12f44de4de80d92807f9d35c2cbc33e/py3Dmol-2.1.0-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


- Provide location for downloaded .pdb file and let's take a look at it:

In [4]:
import py3Dmol

AldO = '../pMPNN_analysis/examples/pdbs/input/2vfs.pdb' # define location of the pdb file

view = py3Dmol.view(width=400, height=300)
AldO_pdb = open(AldO).read()
view.addModel(AldO_pdb, 'pdb')
view.setBackgroundColor('white')

protein = {'chain':'A'}
selection = {'resn':'FAD', 'chain':'A', 'byres':'true', 'expand': 6}
view.setStyle({'chain': 'A'}, {'cartoon': {'color': 'white'}})  # Cartoon representation for chain A (protein)
view.setStyle({'resn': 'FAD'},{'stick': {'colorscheme': 'greenCarbon'}})
view.setStyle({'resn': 'XYL'},{'stick': {'colorscheme': 'pinkCarbon'}})
view.zoomTo()
view.show()

- We see AldO as a white cartoon, FAD as green and XYL as pink sticks. We can very roughly (without any literature research) define an active site as residues surrounding a substrate XYL and cofactor FAD. But this is not our protein of interest! What we will do next, is we will download an AlphaFol2 predicted structure of HotAldO and simply insert FAD into the .pdb file of HotAldO. 

- **Warning**: this is an acceptable approach if two homologues share (very) high sequence and structure similarity. It it is not the case, one should properly dock a cofactor and a substrate into an active site of a protein of interest (POI). AldO is nearly identical to HotAldO in the active site, so we don't do docking here. 

- HotAldO AlphaFold2 (AF2) prediction can be found here: https://alphafold.ebi.ac.uk/entry/A0LST6

- Download AF2 prediction and transpose coordinates of FAD and XYL into .pdb file of HotAldO. This can be done either in PyMOL, Chimera or e.g. in commanline using cat and grep commands. 

- Let's now take a look at our corresponding HotAldO with bound FAD and XYL:

In [ ]:
HotAldO = '../pMPNN_analysis/examples/pdbs/input/HotAldO_FAD_XYL.pdb'
view = py3Dmol.view(width=400, height=300)
HotAldO_pdb = open(HotAldO).read()
view.addModel(HotAldO_pdb, 'pdb')
view.setBackgroundColor('white')

protein = {'chain':'A'}
view.setStyle({'chain': 'A'}, {'cartoon': {'color': 'cornflowerblue'}})  # Cartoon representation for chain A (protein)
view.setStyle({'resn': 'FAD'},{'stick': {'colorscheme': 'greenCarbon'}})
view.setStyle({'resn': 'XYL'},{'stick': {'colorscheme': 'pinkCarbon'}})
view.zoomTo()
view.show()

- HotAldO is depicted now as a blue cartoon. Perhaps there are some steric clashes with FAD, so docking might be a good idea in this case as well. But since we only care about residues 6 $\AA$ around our cofactor in this "frankenstein" HotAldO, it is acceptable. 

- Now how does the selection of residues 6 $\AA$ around FAD looks like?

In [ ]:
selection = {'resn':'FAD', 'chain':'A', 'byres':'false', 'expand': 6}
view.setStyle(selection,{'stick':{'colorscheme':'lightgreyCarbon'}})
view.setStyle({'resn': 'FAD'},{'stick': {'colorscheme': 'greenCarbon'}})
view.setStyle({'resn': 'XYL'},{'stick': {'colorscheme': 'pinkCarbon'}})

### 4. Fixing residues

- We now have familiriaze ourselves with our protein of interest. Let's finally fix residues which are going to be retained in pMPNN design. We will use a script developed for this purpose (available as CLI as well). 

- for myself: think here about location of scripts in the github repo

In [ ]:
from fix_res_select import *

```fix_res_select``` contains a class **res_selection** which defines a function **fix_residues**. We will use it to find residues positions in 6 $\AA$ around FAD and XYL:

In [ ]:
test = res_selection(HotAldO, ['FAD', 'XYL']) #definition of the class; first arg = loc of a pdb, second = list of HETATMs as str
fixed_res = test.fix_residues(6)

print('number of residues we will fix:', len(fixed_res))
print('residue numbers we will fix:')
for i in fixed_res:
	print(i, end=' ')

print('\nlength of HotAldO is 417 => we fix', round(len(fixed_res)/417*100,0), 'percent of residues')

- This string of residue numbers separated by a space is an input for pMPNN design script to fix residues. Now we have a .pdb file of HotAldO and positions we want to fix. The design script which was used to generate sequences with pMPNN can be found in examples folder. 

- I won't go to any details on all parameters pMPNN uses during inference, the github repo of pMPNN has a great documentation of all modii operandi: https://github.com/dauparas/ProteinMPNN/tree/main

- I designed 150 sequences (default = 8) with sampling T = 0.1 (less diversity) using soluble model weights. You can experiment with different settings!  

### 5. Understanding fasta output of ProteinMPNN

- .fa file which is the output of pMPNN contains wildtype sequence and specificed number of designed variants with some metadata. Let's take a look at the structure of a fasta output:

In [ ]:
HotAldO_des_fa = '../pMPNN_analysis/examples/fasta/sol_HotAldO.fa' # define location of the fasta file

with open(HotAldO_des_fa, 'r') as fasta:
	lines = fasta.readlines()
lines[:4] # first 4 lines of the fasta file

- We will use all the metadata in our downstream analysis. What important now is that each uneven line is a sequence while headers are evenly numbered including 0 index. We can easily extract sequences:

In [ ]:
all_seqs = [lines[i].strip('\n') for i in range(len(lines)) if i %2 != 0]
orig_seq = all_seqs[0]
orig_seq # original sequence

- By having a .fa file with sequences, we can fold all of them using **ESMfold** (doi: 10.1126/science.ade2574). The script used to fold all 150 variants and native HotAldO can be found in examples folder. 

### 6. Analyzing pMPNN-designed sequences and interpreting the results


Now the fun stuff begins. We will use several hand-coded scripts to extract interesting data from .fa and structures of pMPNN-designed variants and then will visualize all the results in some plots. 

- In the first part we will extract plddt values of predictions, rmsd to native, will compute some biophysical parameters and then take a look at the dataset we generated.

- We will need to provide a .fa file with designed sequences, folder with .pdb files of predicted sequences

In [ ]:
from mpnn_analysis import *

```mpnn_analysis``` contains a class **mpnn_analysis** which processess results from .fa and .pdb (predictions) files and aggregates them in one data set. For more details, read documentation of all the functions

In [ ]:
HotAldO_des_fa = '../pMPNN_analysis/examples/fasta/sol_HotAldO.fa'
des_predictions_ESMfold = '../pMPNN_analysis/examples/pdbs/ESMfold_pred' # define location of the ESMfold predictions

In [ ]:
HotAldO_analysis = mpnn_analysis(HotAldO_des_fa, None, des_predictions_ESMfold, str_predict='ESMfold')
data_HotAldO_des = HotAldO_analysis.aggregate_data() # this method will conduct analyses on sequences and structures and return them in a pandas dataframe

- We defined a class and then executed a function which aggregates data in one data set. What data is there? We extracted sequence recovery & identity, plddt and some interesting biophysical metrics. Among those are: fraction of exposed designed residues, fraction of hydrophilic exposed residues, total charge of proteins and their core sizes. 

- The latter metrics are approximations and have to be inspected with caution. Core size is the number of solvent-excluded residues; charges are approximated by taking only negatively/positively charged residue types and ignoring microenvironment. 

- Let's take a look at the dataset we generated:

In [ ]:
print('rows, columns:', data_HotAldO_des.shape)
data_HotAldO_des.head()

- I have prepared a class to plot interesting properties we extracted and some additional metrics explaining how ProteinMPNN works (with pre-loaded soluble weights in this case)

In [ ]:
from plot_mpnn_analysis import *

In [ ]:
figs_out = '../pMPNN_analysis/examples/figs_out' # define location of the output figures
plotting_class = plot_mpnn_analysis(data_HotAldO_des, HotAldO_des_fa, None, des_predictions_ESMfold, figs_out = figs_out, str_predict='ESMfold')

- Please, open documentation of the class to understand what functions do what; for now I will only show the most useful ones. 

- First, let's plot sequence recovery and sequence identity: 

In [ ]:
plotting_class.plot_seq_recovery(save='no')

- Reported native sequence recovery of pMPNN is around 51%, so it performs in our case as expected. Though we have fixed quite some residues from the native protein, therefore sequence identity to the native is a more interesting metric. We see that on avg designed variants are nearly 60% identical to wt. 

- Let's now analyze what common residue exchanges we can find. We will plot a 20x20 heatmap with original residue type on x-axis and designed residue type on y-axis: 

In [ ]:
plotting_class.heatmap_exchanged_residues(sort_hydrophobic=True, non_wt_only=False, save='no', )

- This function iteratively counts what residues appear at each of the native sequence positions from the .fa file. Diagonal in this case represents "wt-wt exchange" meaning pMPNN retains these residues from native HotAldO. It turns out that GLY, PRO and LEU are the most commonly retained residues! This does make sense since GLY and PRO are important for structural integrity of protein folds: they introduce turns and kinks. Keeping them at wildtype positions should guarantee that the protein adopts the desired fold.

- Interestingly, (I digged into it) at one position pMPNN mutated K -> V in many designed sequences, so it stands out if one normalizes to the total exchanges of lysines. We can say that pMPNN actually doesn't like to touch lysines at all! 

- Maybe now let's consider only wt-non wt exchanges, so that diagonal is zero:

In [ ]:
plotting_class.heatmap_exchanged_residues(sort_hydrophobic=True, non_wt_only=True, save='no')

- We count here number of exchanges, so it's sensitive to the occurence of the residue type in the native sequence. We see that pMPNN decided to very frequently mutate ARG to GLU. This changes the charge of the protein by 2 digits per each such mutation!

- What if we now try to plot charge of the designed solvent exposed residues?

In [ ]:
plotting_class.plot_charges(selection = 'exposed', save='no', )

- ```plot_charges``` first computes positions where pMPNN mutated original sequence for all 150 designs. Then it subsets only those designed residues that are solvent exposed (based on SASA values) and then computes their charge. That means we have only one protein for "input" label and 150 different for design. We are comparing iteratively residue indices from native HotAldO to each of the corresponding redesigned indices in 150 designs.

- It turns out that pMPNN with soluble pre-loaded weights rendered surface of designs negatively charged.

- Now let's take a look at the total charge distribution of our designs:

In [ ]:
plotting_class.plot_charges(selection = 'total', save='no', )

- Indeed, as expected, designs possess higher negative charge than the wildtype.

- Another metric we can investigate is hydrophilicity of solvent-exposed designed residues. Perhaps we would expect if sol pMPNN negatively charges surfaces that our solvent-exposed regions might have become more hydrophilic ..?

In [ ]:
plotting_class.violin_hydrophilic_exposed(save='no')

- Among the redesigned solvent-exposed residues, hydrophilic residues dominate.

### 7. Conclusions


This tutorial introduced the pipeline of pMPNN and suggested metrics to analyze when redesigning enzyme for the improved solubility. Our analyses showed that sol pMPNN (i) rendered protein surfaces more hydrophilic, (ii) negatively charged, and (iii) that we can find distinct preferred introduced mutations in the heatmaps. 

Credits: Seva Viliuga